In [1]:
import pandas as pd
from collections import Counter
import ast
import numpy as np
import matplotlib.pyplot as plt
import datetime
# from skimage.io import imread
import os
import statistics
from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsClassifier

In [2]:
train_data = pd.read_csv('final_train_data.csv')

In [3]:
train_data=train_data[train_data['Pleural Effusion'].notna()]

In [4]:
train_data

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,...,corner_193,corner_194,corner_195,corner_196,corner_197,corner_198,corner_199,Right_Lung_Intensity,Left_Lung_Intensity,Asymmetry
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,...,0,0,0,0,0,0,0,110.128293,96.570501,0.596201
5,CheXpert-v1.0-small/train/patient00004/study1/...,Female,20,Frontal,PA,1.0,0.0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,80.388532,75.504396,0.270156
6,CheXpert-v1.0-small/train/patient00004/study1/...,Female,20,Lateral,NaN,1.0,0.0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN
7,CheXpert-v1.0-small/train/patient00005/study1/...,Male,33,Frontal,PA,1.0,NaN,0.0,NaN,NaN,...,0,0,0,0,0,0,0,70.260511,79.038964,0.225228
8,CheXpert-v1.0-small/train/patient00005/study1/...,Male,33,Lateral,NaN,1.0,NaN,0.0,NaN,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223403,CheXpert-v1.0-small/train/patient64533/study1/...,Male,75,Frontal,AP,NaN,NaN,1.0,NaN,NaN,...,0,0,0,0,0,0,0,95.301345,94.698435,0.285247
223405,CheXpert-v1.0-small/train/patient64534/study1/...,Male,63,Frontal,AP,NaN,NaN,NaN,1.0,NaN,...,0,0,0,0,0,0,0,64.932217,61.532157,0.267634
223407,CheXpert-v1.0-small/train/patient64536/study2/...,Female,61,Frontal,AP,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,143.463986,76.111103,0.727078
223409,CheXpert-v1.0-small/train/patient64537/study2/...,Male,59,Frontal,AP,NaN,NaN,NaN,-1.0,NaN,...,0,0,0,0,0,0,0,62.346834,91.663944,0.368473


In [5]:
# One-hot encode the categorical columns
train_data = pd.get_dummies(train_data, columns=['Sex'])
train_data = pd.get_dummies(train_data, columns=['Frontal/Lateral'])
train_data = pd.get_dummies(train_data, columns=['AP/PA'])

In [6]:
train_data

,Path,Age,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,...,Asymmetry,Sex_Female,Sex_Male,Sex_Unknown,Frontal/Lateral_Frontal,Frontal/Lateral_Lateral,AP/PA_AP,AP/PA_LL,AP/PA_PA,AP/PA_RL
1,CheXpert-v1.0-small/train/patient00002/study2/...,87,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,...,0.596201,True,False,False,True,False,True,False,False,False
5,CheXpert-v1.0-small/train/patient00004/study1/...,20,1.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,...,0.270156,True,False,False,True,False,False,False,True,False
6,CheXpert-v1.0-small/train/patient00004/study1/...,20,1.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,True,False,False,False,True,False,False,False,False
7,CheXpert-v1.0-small/train/patient00005/study1/...,33,1.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,...,0.225228,False,True,False,True,False,False,False,True,False
8,CheXpert-v1.0-small/train/patient00005/study1/...,33,1.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,...,NaN,False,True,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223403,CheXpert-v1.0-small/train/patient64533/study1/...,75,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,...,0.285247,False,True,False,True,False,True,False,False,False
223405,CheXpert-v1.0-small/train/patient64534/study1/...,63,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,0.267634,False,True,False,True,False,True,False,False,False
223407,CheXpert-v1.0-small/train/patient64536/study2/...,61,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,0.727078,True,False,False,True,False,True,False,False,False
223409,CheXpert-v1.0-small/train/patient64537/study2/...,59,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,...,0.368473,False,True,False,True,False,True,False,False,False


In [15]:
from sklearn.metrics import accuracy_score, brier_score_loss
import numpy as np

def compute_calibration_metrics(y_true, y_prob, n_bins=5):
    """
    Computes Accuracy, ECE, MCE, and Brier Score for multiclass classification.

    Parameters:
    - y_true: 1D array of true class labels
    - y_prob: 2D array of predicted probabilities (n_samples, n_classes)
    - n_bins: Number of bins for calibration (default 5)

    Returns:
    - accuracy: Classification accuracy
    - ece: Expected Calibration Error
    - mce: Maximum Calibration Error
    - brier: Brier Score (one-vs-rest averaged)
    """
    conf = np.max(y_prob, axis=1)
    pred = np.argmax(y_prob, axis=1)
    accs = (pred == y_true).astype(float)
    accuracy = accs.mean()

    bin_edges = np.linspace(0, 1, n_bins + 1)
    idx = np.digitize(conf, bin_edges, right=True) - 1
    idx = np.clip(idx, 0, n_bins - 1)

    ece = mce = 0.0
    for i in range(n_bins):
        mask = idx == i
        if not mask.any():
            continue
        bin_acc  = accs[mask].mean()
        bin_conf = conf[mask].mean()
        weight   = mask.sum() / len(y_true)
        gap = abs(bin_acc - bin_conf)
        ece += weight * gap
        mce = max(mce, gap)

    brier = np.mean([
        brier_score_loss((y_true == i).astype(int), y_prob[:, i])
        for i in range(y_prob.shape[1])
    ])

    return accuracy, ece, mce, brier

In [51]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

def prepare_data(train_data, feature_cols, test_size=0.2, random_state=42, 
                 use_pca=False, pca_components=0.80):
    """
    Prepares data for XGBoost, applying PCA only to 'hist*' and 'corner*' features if enabled.

    Parameters:
    - train_data: DataFrame with all features and target column 'Pleural Effusion'
    - feature_cols: List of feature column names
    - test_size: Proportion of data for testing
    - random_state: Random seed
    - use_pca: Whether to apply PCA to 'hist*' and 'corner*' features
    - pca_components: Number of PCA components or explained variance threshold

    Returns:
    - X_train, X_test: Feature arrays
    - y_train_encoded, y_test_encoded: Encoded targets
    - encoder: Fitted LabelEncoder
    - scaler: Fitted StandardScaler (if PCA used)
    - pca_model: Fitted PCA object (if PCA used)
    """

    if not isinstance(train_data, pd.DataFrame):
        raise ValueError("train_data must be a pandas DataFrame")
    if not all(col in train_data.columns for col in feature_cols):
        raise ValueError("Some feature_cols are not found in train_data")

    # Drop rows with missing target
    train_data = train_data.dropna(subset=['Pleural Effusion'])

    # Extract features and target
    X = train_data[feature_cols].copy()
    y = train_data['Pleural Effusion']

    # Split into train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )

    # Encode labels
    encoder = LabelEncoder()
    y_train_encoded = encoder.fit_transform(y_train)
    y_test_encoded = encoder.transform(y_test)

    # Identify PCA-eligible features
    pca_features = [col for col in feature_cols if col.startswith("hist") or col.startswith("corner")]
    non_pca_features = [col for col in feature_cols if col not in pca_features]

    scaler = None
    pca_model = None

    if use_pca and pca_features:
        # Standardize PCA features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train[pca_features])
        X_test_scaled = scaler.transform(X_test[pca_features])

        # Apply PCA
        pca_model = PCA(n_components=pca_components)
        X_train_pca = pca_model.fit_transform(X_train_scaled)
        X_test_pca = pca_model.transform(X_test_scaled)

        print(f"PCA: Retained {pca_model.n_components_} components explaining "
              f"{100 * pca_model.explained_variance_ratio_.sum():.1f}% variance")

        # Combine transformed + untouched features
        X_train_final = np.hstack([X_train[non_pca_features].values, X_train_pca])
        X_test_final = np.hstack([X_test[non_pca_features].values, X_test_pca])
    else:
        X_train_final = X_train.values
        X_test_final = X_test.values

    return X_train_final, X_test_final, y_train_encoded, y_test_encoded, encoder, scaler, pca_model


In [43]:
def run_xgboost_cv(X, y, n_splits=5, random_state=42):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    aucs, accs, eces, mces, briers = [], [], [], [], []

    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y), start=1):
        print(f"\n🔄 Fold {fold}/{n_splits}")
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss',
                              objective='multi:softprob', num_class=3,
                              random_state=random_state)
        model.fit(X_train, y_train)

        y_prob = model.predict_proba(X_test)
        auc = roc_auc_score(y_test, y_prob, multi_class='ovo')
        acc, ece, mce, brier = compute_calibration_metrics(y_test, y_prob)

        print(f"AUC: {auc:.3f} | ACC: {acc:.3f} | ECE: {ece:.4f} | MCE: {mce:.4f} | BRIER: {brier:.4f}")
        aucs.append(auc)
        accs.append(acc)
        eces.append(ece)
        mces.append(mce)
        briers.append(brier)

    print("\n📊 AVERAGE METRICS:")
    print(f"AUC:   {np.mean(aucs):.3f}")
    print(f"ACC:   {np.mean(accs):.3f}")
    print(f"ECE:   {np.mean(eces):.4f}")
    print(f"MCE:   {np.mean(mces):.4f}")
    print(f"Brier: {np.mean(briers):.4f}")

    return {
        'auc': np.mean(aucs),
        'acc': np.mean(accs),
        'ece': np.mean(eces),
        'mce': np.mean(mces),
        'brier': np.mean(briers)
    }

In [44]:
train_data

,Path,Age,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,...,Asymmetry,Sex_Female,Sex_Male,Sex_Unknown,Frontal/Lateral_Frontal,Frontal/Lateral_Lateral,AP/PA_AP,AP/PA_LL,AP/PA_PA,AP/PA_RL
1,CheXpert-v1.0-small/train/patient00002/study2/...,87,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,...,0.596201,True,False,False,True,False,True,False,False,False
5,CheXpert-v1.0-small/train/patient00004/study1/...,20,1.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,...,0.270156,True,False,False,True,False,False,False,True,False
6,CheXpert-v1.0-small/train/patient00004/study1/...,20,1.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,True,False,False,False,True,False,False,False,False
7,CheXpert-v1.0-small/train/patient00005/study1/...,33,1.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,...,0.225228,False,True,False,True,False,False,False,True,False
8,CheXpert-v1.0-small/train/patient00005/study1/...,33,1.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,...,NaN,False,True,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223403,CheXpert-v1.0-small/train/patient64533/study1/...,75,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,...,0.285247,False,True,False,True,False,True,False,False,False
223405,CheXpert-v1.0-small/train/patient64534/study1/...,63,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,0.267634,False,True,False,True,False,True,False,False,False
223407,CheXpert-v1.0-small/train/patient64536/study2/...,61,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,0.727078,True,False,False,True,False,True,False,False,False
223409,CheXpert-v1.0-small/train/patient64537/study2/...,59,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,...,0.368473,False,True,False,True,False,True,False,False,False


In [45]:
train_data.columns

Index(['Path', 'Age', 'No Finding', 'Enlarged Cardiomediastinum',
       'Cardiomegaly', 'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation',
       'Pneumonia',
       ...
       'Asymmetry', 'Sex_Female', 'Sex_Male', 'Sex_Unknown',
       'Frontal/Lateral_Frontal', 'Frontal/Lateral_Lateral', 'AP/PA_AP',
       'AP/PA_LL', 'AP/PA_PA', 'AP/PA_RL'],
      dtype='object', length=484)

# TRAIN AND TEST WITHOUT CLINICAL FEATURES

In [46]:
# Define the subset of columns for an experiment
# Correct way to drop multiple columns
# train_data_no_clinical = train_data.drop(['Pleural Effusion', 'Path','Right_Lung_Intensity', 'Left_Lung_Intensity', 'Asymmetry', ], axis=1)
# feature_columns = train_data_no_clinical.columns  # Modify as needed
fixed_columns = ['Age', 'Sex_Male', 'Sex_Female', 'Sex_Unknown']

# Select columns that start with 'corner' or 'hist'
pattern_columns = [col for col in train_data.columns if col.startswith('corner') or col.startswith('hist')]

# Combine all desired columns
feature_columns = fixed_columns + pattern_columns
# Prepare the data
X_train, X_test, y_train, y_test,*_ = prepare_data(train_data, feature_columns, use_pca=False)

# Run XGBoost experiment
model, auc, acc, ece, mce, brier = run_xgboost_experiment(X_train, y_train, X_test, y_test)


/opt/anaconda3/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:28:56] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


AUC:    0.653
Acc:    0.696
ECE:    0.0136
MCE:    0.1031
Brier:  0.1436


# TRAIN AND TEST WITH CLINICAL FEATURES

In [47]:
fixed_columns = ['Path','Age', 'Sex_Male', 'Sex_Female', 'Sex_Unknown', 'Pleural Effusion']
# Select columns that start with 'corner' or 'hist'
pattern_columns = [col for col in train_data.columns if col.startswith('corner') or col.startswith('hist')]

# Combine all desired columns
feature_columns_drop = fixed_columns + pattern_columns

feature_columns = train_data.drop(feature_columns_drop, axis=1).columns

# Prepare the data
X_train, X_test, y_train, y_test,*_ = prepare_data(train_data, feature_columns, use_pca=False)

# Run XGBoost experiment
model, auc, acc, ece, mce, brier = run_xgboost_experiment(X_train, y_train, X_test, y_test)

/opt/anaconda3/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:29:38] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


AUC:    0.869
Acc:    0.849
ECE:    0.0060
MCE:    0.0909
Brier:  0.0787


# TRAIN AND TEST WITH ALL FEATURES

In [48]:
# Define the subset of columns for an experiment
# Correct way to drop multiple columns
train_data_clinical = train_data.drop(['Pleural Effusion', 'Path'], axis=1)
feature_columns = train_data_clinical.columns  # Modify as needed


X_train, X_test, y_train, y_test,*_ = prepare_data(train_data, feature_columns, use_pca=False)

# Run XGBoost experiment
model, auc, acc, ece, mce, brier = run_xgboost_experiment(X_train, y_train, X_test, y_test)


/opt/anaconda3/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:29:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


AUC:    0.865
Acc:    0.847
ECE:    0.0076
MCE:    0.0520
Brier:  0.0794


# PCA

pca with full

In [53]:
# Define the subset of columns for an experiment
# Correct way to drop multiple columns
train_data_clinical = train_data.drop(['Pleural Effusion', 'Path'], axis=1)
feature_columns = train_data_clinical.columns  # Modify as needed


X_train, X_test, y_train, y_test,*_ = prepare_data(train_data, feature_columns, use_pca=True)

# Run XGBoost experiment
model, auc, acc, ece, mce, brier = run_xgboost_experiment(X_train, y_train, X_test, y_test)


PCA: Retained 105 components explaining 80.0% variance


/opt/anaconda3/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:34:37] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


AUC:    0.867
Acc:    0.847
ECE:    0.0080
MCE:    0.0371
Brier:  0.0791


pca no clinical

In [52]:
# Define the subset of columns for an experiment
# Correct way to drop multiple columns
# train_data_no_clinical = train_data.drop(['Pleural Effusion', 'Path','Right_Lung_Intensity', 'Left_Lung_Intensity', 'Asymmetry', ], axis=1)
# feature_columns = train_data_no_clinical.columns  # Modify as needed
fixed_columns = ['Age', 'Sex_Male', 'Sex_Female', 'Sex_Unknown']

# Select columns that start with 'corner' or 'hist'
pattern_columns = [col for col in train_data.columns if col.startswith('corner') or col.startswith('hist')]

# Combine all desired columns
feature_columns = fixed_columns + pattern_columns
# Prepare the data
X_train, X_test, y_train, y_test,*_ = prepare_data(train_data, feature_columns, use_pca=True)

# Run XGBoost experiment
model, auc, acc, ece, mce, brier = run_xgboost_experiment(X_train, y_train, X_test, y_test)


PCA: Retained 105 components explaining 80.0% variance


/opt/anaconda3/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:34:05] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


AUC:    0.644
Acc:    0.687
ECE:    0.0147
MCE:    0.0242
Brier:  0.1470
